In [ ]:
# import moviepy.editor as mp
# from pydub import AudioSegment
# from pydub.silence import split_on_silence
# from vosk import Model, KaldiRecognizer, SetLogLevel
# import numpy as np

# # Set log level to suppress Vosk log messages
# SetLogLevel(0)

# # Step 1: Extract audio from video
# video = mp.VideoFileClip("um.mp4")
# audio = video.audio

# # Step 2: Convert audio to WAV for speech recognition
# audio.export("audio.wav", format="wav", codec="pcm_s16le")

# # Step 3: Speech recognition using Vosk
# model_path = "../vosk-model-en-us-0.22"
# try:
#     model = Model(model_path)
#     print("Model loaded successfully.")
# except Exception as e:
#     print(f"Error loading model: {e}")
#     # Handle the error or exit the program

# # Step 4: Process audio chunks
# audio_data = AudioSegment.from_wav("audio.wav")
# chunks = split_on_silence(audio_data, silence_thresh=-40)
# recognized_text = ""

# for chunk in chunks:
#     # Convert audio data to numpy array of 16-bit signed integers
#     audio_np = np.array(chunk.get_array_of_samples(), dtype=np.int16)
#     recognizer = KaldiRecognizer(model, audio_np)
#     recognized_text += recognizer.FinalResult()

# # Step 5: Identify "um" occurrences and cut corresponding video segments
# um_segments = [(start, end) for start, end in zip(chunks[:-1], chunks[1:]) if "um" in recognized_text[start:end]]

# edited_segments = []
# for start, end in um_segments:
#     edited_segments.append(video.subclip(start / 1000, end / 1000))  # Convert milliseconds to seconds

# final_video = mp.concatenate_videoclips(edited_segments)

# # Step 6: Export final video
# final_video.write_videofile("umRemove.mp4", codec="libx264", audio_codec="aac")


MoviePy - Writing audio in audio.wav


MoviePy - Done.
Model loaded successfully.
[18770, 17990, -28026, 117, 16727, 17750, 28006, 8308, 16, 0]
0.0755152020121398


In [9]:
import moviepy.editor as mp
from pydub import AudioSegment
from pydub.silence import split_on_silence
import wit
import requests

# Set up Wit.ai
access_token = "G5VWND33DLPK23JLTMI5K2H25EYB747W"

wit.access_token = access_token

# # Set up MoviePy
# SetLogLevel(0)

# Step 1: Extract audio from video
video = mp.VideoFileClip("um.mp4")
audio = video.audio

# Step 2: Convert audio to WAV for speech recognition
audio.write_audiofile("audio.wav", codec="pcm_s16le")

# Step 3: Process audio chunks
audio_data = AudioSegment.from_wav("audio.wav")
chunks = split_on_silence(audio_data, silence_thresh=-40)
recognized_text = ""

for chunk in chunks:
    # Convert audio data to numpy array of 16-bit signed integers
    audio_np = chunk.get_array_of_samples()

    # Upload the audio to Wit.ai for recognition
    wit_url = "https://api.wit.ai/speech"
    headers = {
        "Authorization": f"Bearer {access_token}",
        "Content-Type": "audio/wav",
    }
    response = requests.post(wit_url, headers=headers, data=chunk.raw_data)

    # Extract recognized text from the response
    recognized_text += response.json().get("_text", "")

# Step 4: Identify "um" occurrences and cut corresponding video segments
um_segments = [(start, end) for start, end in zip(chunks[:-1], chunks[1:]) if "um" in recognized_text[start:end]]

edited_segments = []
for start, end in um_segments:
    edited_segments.append(video.subclip(start / 1000, end / 1000))  # Convert milliseconds to seconds

final_video = mp.concatenate_videoclips(edited_segments)

# Step 5: Export final video
final_video.write_videofile("umRemove.mp4", codec="libx264", audio_codec="aac")

# Clean up Wit.ai
wit.close()


MoviePy - Writing audio in audio.wav


MoviePy - Done.


TypeError: slice indices must be integers or None or have an __index__ method

In [ ]:
# from ibm_watson import SpeechToTextV1
# from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

# authenticator = IAMAuthenticator("YOUR_API_KEY")
# speech_to_text = SpeechToTextV1(authenticator=authenticator)

# audio_file = "path/to/audio/file.wav"

# with open(audio_file, "rb") as audio_file:
#     result = speech_to_text.recognize(audio=audio_file, content_type="audio/wav")
#     print("Transcript: {}".format(result.result["results"][0]["alternatives"][0]["transcript"]))
